In [ ]:
#!pip install -U jedi-language-server
#!pip install nltk
#!pip install nltk spacy
#!python -m spacy download de_core_news_sm

---

# 2 Datenverarbeitung

### 2.1 Einlesen und Vorbereiten des Datensatzes

In [1]:
import pandas as pd
filepath = "news.csv"

#----- load dataset -----
def load_data(filepath):
    data = pd.read_csv(filepath, encoding='utf-8')
    data = data[['Body', 'Fake']]
    data.columns = ['text', 'label']
    return data

data = load_data('news.csv')
print(f"Datensatz gesamt: {len(data)}\n{data['label'].value_counts()}")

Datensatz gesamt: 63868
label
0    59241
1     4627
Name: count, dtype: int64


In [66]:
data.head()

,text,label
0,Die Sondierungsgespräche zwischen Union und SP...,1
1,Nun ist es auch medizinisch offiziell bestätig...,1
2,"Es waren zähe Verhandlungen, doch die Freien D...",1
3,Wo treibt sie sich immer bis spät in die Nacht...,1
4,Der Parteivorstand drückt nochmal ein Auge zu:...,1


### 2.2 Datensätze reduzieren

In [59]:
#----- redunce dataset size -----
def balance_all_data():
    fake_news = data[data['label'] == 1]
    true_news = data[data['label'] == 0].sample(n=len(fake_news), random_state=42)  

    combined = pd.concat([true_news, fake_news]).sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Reduzierter Datensatz gesamt: {len(combined)}\n{combined['label'].value_counts()}")  
    return combined

balanced_data = balance_all_data()

Reduzierter Datensatz gesamt: 9254
label
0    4627
1    4627
Name: count, dtype: int64


### 2.3 Textaufbereitung

In [68]:
import pandas as pd
import re
import nltk
import spacy
from nltk.corpus import stopwords

#----- configs -----
nlp = spacy.load("de_core_news_sm")

#nltk.download('stopwords')
stop_words = set(stopwords.words('german'))


#----- main preprocession -----
def preprocess_text(text):
    #1)
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    
    #2)
    text = text.lower()
    
    #3)
    doc = nlp(text) #tokenisierung mit spacy
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]

    #4)
    adjective_count = sum(1 for token in doc if token.pos_ == "ADJ")
    adverb_count = sum(1 for token in doc if token.pos_ == "ADV")
    entity_count = len(doc.ents)
    
    return " ".join(tokens), adjective_count, adverb_count, entity_count

balanced_data[['text_clean', 'adjective_count', 'adverb_count', 'entity_count']] = balanced_data['text'].apply(
    lambda x: pd.Series(preprocess_text(str(x)))
)

balanced_data.head()

,text,label,text_clean,adjective_count,adverb_count,entity_count
0,Neuartige Übertragungsmethode von Neurostimula...,0,neuartig Übertragungsmethode Neurostimulation ...,36,29,9
1,"Viele hatten es schon lange geahnt, jetzt ist ...",1,viele schon lange ahnen endlich amtlich Verfas...,18,40,16
2,Der Zar lässt die Säbel rasseln: Vor dem Hinte...,1,zar Lässt säbel Rassel Hintergrund zunehmend S...,15,27,20
3,Pumpt E10 unter die Erde: Bohrturm\r\n\r\nIm Z...,1,pumpen e Erde bohrturm \r\n\r\n Zug sogenannte...,11,23,11
4,iPad 2 wird angeblich mit extrem hoher Auflösu...,0,ipad angeblich extrem hoch Auflösung vorlege...,21,53,22


---

# 3 Feature Engineering

### 3.1 Merkmalsextraktion

In [95]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def create_feature_df(vectorizer):
    X_text = vectorizer.fit_transform(balanced_data['text_clean']).toarray()

    X_text_df = pd.DataFrame(X_text, columns=vectorizer.get_feature_names_out())
    X_other_df = balanced_data[['adjective_count', 'adverb_count', 'entity_count']].reset_index(drop=True)

    return pd.concat([X_text_df, X_other_df], axis=1)

# ----- feature extraction -----
vectorizer = CountVectorizer(max_features=1000)
X = create_feature_df(vectorizer)
y = balanced_data['label']

In [96]:
X

,ab,abend,abgeordneter,absolut,acht,afd,ag,aktie,aktion,aktiv,...,österreich,österreichisch,övp,überhaupt,übernehmen,überraschend,überzeugen,adjective_count,adverb_count,entity_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,36,29,9
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,18,40,16
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15,27,20
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,23,11
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21,53,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,16,30,11
9250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16,14,18
9251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17,20,10
9252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15,16,15


---

# 4 Modelltraining

### 4.1 Datenaufteilung und 4.2 Training des Random Forest Classifiers

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# ----- Modelltraining -----
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

---

# 5 Evaluation und Tests

In [101]:
from sklearn.metrics import accuracy_score
import joblib 

# ----- Genauigkeit -----
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Genauigkeit: {accuracy:.2f}")

# ----- Modell speichern -----
joblib.dump(model, "model/fake_news_model.pkl")
joblib.dump(vectorizer, "model/vectorizer.pkl")

print("Modell gespeichert!")

Genauigkeit: 0.96
Modell gespeichert!


---

In [99]:
import joblib
import numpy as np
import pandas as pd

red_c = "\033[91m["
green_c = "\033[92m["
black_c = "]\033[0m"

# ----- Modell laden -----
model = joblib.load("model/fake_news_model.pkl")
vectorizer = joblib.load("model/vectorizer.pkl")

test_news = [
    #3 echte Nahrichten
    "GPS.AT übernimmt Grazer Unternehmen XLOC Tracking Systems und setzt Wachstumskurs auch 2015 fort",
    "TechFirma übernimmt führendes Unternehmen im Bereich Künstliche Intelligenz und setzt globales Expansionsziel 2025 fort.",
    "EnergieRiese investiert massiv in erneuerbare Technologien und plant Expansion in den asiatischen Markt.",
    #3 gefälschte Nachrichten
    "Schockierender Bericht: Politiker kontrollieren das Wetter!",
    "Prominente warnen: Die Erde ist in Wahrheit flach!",
    "Wissenschaftler entdecken Heilmittel gegen Krebs – aber Pharmaindustrie blockiert es!",
]

# ----- Fake-News-Klassifikation -----
def predict_news(text):
    text_cleaned, adj, adv, ent = preprocess_text(text)
    
    text_vectorized = vectorizer.transform([text_cleaned]).toarray()
    text_df = pd.DataFrame(text_vectorized, columns=vectorizer.get_feature_names_out())
    features_df = pd.DataFrame([[adj, adv, ent]], columns=['adjective_count', 'adverb_count', 'entity_count'])
    combined =  pd.concat([text_df, features_df], axis=1)
    
    prediction = model.predict(combined)[0]
    return "Fake" if prediction == 1 else "Echt"


results = [(text, predict_news(text)) for text in test_news]
for text, prediction in results:
    
    if prediction == "Echt":
        print(f"{green_c}{prediction}{black_c} {text}")
    else:
        print(f"{red_c}{prediction}{black_c} {text}")

[Echt] GPS.AT übernimmt Grazer Unternehmen XLOC Tracking Systems und setzt Wachstumskurs auch 2015 fort
[Echt] TechFirma übernimmt führendes Unternehmen im Bereich Künstliche Intelligenz und setzt globales Expansionsziel 2025 fort.
[Echt] EnergieRiese investiert massiv in erneuerbare Technologien und plant Expansion in den asiatischen Markt.
[Fake] Schockierender Bericht: Politiker kontrollieren das Wetter!
[Fake] Prominente warnen: Die Erde ist in Wahrheit flach!
[Fake] Wissenschaftler entdecken Heilmittel gegen Krebs – aber Pharmaindustrie blockiert es!
